In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

sc = spark.sparkContext

In [3]:
from pyspark.sql.types import * 
import pyspark.sql.functions as F

In [4]:
!pwd

/home/talentum/test-jupyter/test/PairRdd


$$
\Large\text{RDD}
$$

####  Finding the airlines with highest number of flights

In [5]:
rdd = sc.textFile('file:///home/talentum/test-jupyter/test/PairRdd/flights.csv')

In [6]:
rdd = rdd.map(lambda line: line.split(','))
print(rdd.take(5))

[['1', '3', '4', '2003', '2211', 'WN', '335', 'N712SW', '128', '116', '-14', '8', 'IAD', 'TPA', '810', '4', '8', '0', '', '0'], ['1', '3', '4', '926', '1054', 'WN', '1746', 'N612SW', '88', '78', '-6', '-4', 'IND', 'BWI', '515', '3', '7', '0', '', '0'], ['1', '3', '4', '1940', '2121', 'WN', '378', 'N726SW', '101', '87', '11', '25', 'IND', 'JAX', '688', '4', '10', '0', '', '0'], ['1', '3', '4', '1937', '2037', 'WN', '509', 'N763SW', '240', '230', '57', '67', 'IND', 'LAS', '1591', '3', '7', '0', '', '0'], ['1', '3', '4', '754', '940', 'WN', '1144', 'N778SW', '226', '205', '-15', '9', 'IND', 'PHX', '1489', '5', '16', '0', '', '0']]


In [7]:
file_path = 'file:///home/talentum/test-jupyter/test/PairRdd/flights.csv' 

In [8]:
carrierRdd = sc.textFile(file_path)\
    .map(lambda val: val.split(","))\
    .map(lambda column: (column[5], 1))

print(carrierRdd.take(5))

[('WN', 1), ('WN', 1), ('WN', 1), ('WN', 1), ('WN', 1)]


In [9]:
carrierSorted = carrierRdd.reduceByKey(lambda x,y: x+y)\
    .map(lambda a: (a[1], a[0]))\
    .sortByKey(ascending=False)

print(carrierSorted.take(3))

[(356167, 'WN'), (175969, 'AA'), (166445, 'OO')]


#### 2. Determine the most common routes between two cities


In [10]:
file_path = 'file:///home/talentum/test-jupyter/test/PairRdd/airports.csv' 

In [11]:
rdd = sc.textFile(file_path)

rdd = rdd.map(lambda line: line.split(','))
print(rdd.take(10))

[['iata', 'airport', 'city', 'state', 'country', 'lat', 'long'], ['00M', 'Thigpen', 'BaySprings', 'MS', 'USA', '31.95376472', '-89.23450472'], ['00R', 'LivingstonMunicipal', 'Livingston', 'TX', 'USA', '30.68586111', '-95.01792778'], ['00V', 'MeadowLake', 'ColoradoSprings', 'CO', 'USA', '38.94574889', '-104.5698933'], ['01G', 'Perry-Warsaw', 'Perry', 'NY', 'USA', '42.74134667', '-78.05208056'], ['01J', 'HilliardAirpark', 'Hilliard', 'FL', 'USA', '30.6880125', '-81.90594389'], ['01M', 'TishomingoCounty', 'Belmont', 'MS', 'USA', '34.49166667', '-88.20111111'], ['02A', 'Gragg-Wade', 'Clanton', 'AL', 'USA', '32.85048667', '-86.61145333'], ['02C', 'Capitol', 'Brookfield', 'WI', 'USA', '43.08751', '-88.17786917'], ['02G', 'ColumbianaCounty', 'EastLiverpool', 'OH', 'USA', '40.67331278', '-80.64140639']]


In [12]:
cityRdd = sc.textFile(file_path)\
    .map(lambda val: val.split(','))\
    .map(lambda col: (col[0], col[2]))

print(cityRdd.take(5))

[('iata', 'city'), ('00M', 'BaySprings'), ('00R', 'Livingston'), ('00V', 'ColoradoSprings'), ('01G', 'Perry')]


In [13]:
flightOrigDestRdd = sc.textFile(file_path)\
    .map(lambda val: val.split(','))\
    .filter(lambda col: len(col) > 13)\
    .map(lambda col: (col[12], col[13]))

print(flightOrigDestRdd.take(5))

[]


In [14]:
file_path = 'file:///home/talentum/test-jupyter/test/PairRdd/flights.csv' 

In [15]:
flightOrigDestRdd = sc.textFile(file_path).map(lambda val: val.split(',')).filter(lambda col: len(col) >= 13).map(lambda col: (col[12], col[13]))

print(flightOrigDestRdd.take(100))

[('IAD', 'TPA'), ('IND', 'BWI'), ('IND', 'JAX'), ('IND', 'LAS'), ('IND', 'PHX'), ('ISP', 'FLL'), ('ISP', 'FLL'), ('ISP', 'FLL'), ('ISP', 'MCO'), ('ISP', 'MCO'), ('ISP', 'MCO'), ('ISP', 'PBI'), ('ISP', 'PBI'), ('ISP', 'PBI'), ('ISP', 'TPA'), ('JAN', 'HOU'), ('JAN', 'MCO'), ('JAN', 'MDW'), ('JAX', 'BNA'), ('JAX', 'BWI'), ('JAX', 'FLL'), ('JAX', 'FLL'), ('JAX', 'HOU'), ('JAX', 'IND'), ('JAX', 'PHL'), ('JAX', 'PHL'), ('LAS', 'ABQ'), ('LAS', 'AMA'), ('LAS', 'AUS'), ('LAS', 'BNA'), ('LAS', 'BNA'), ('LAS', 'BNA'), ('LAS', 'BUR'), ('LAS', 'BUR'), ('LAS', 'BWI'), ('LAS', 'DEN'), ('LAS', 'DEN'), ('LAS', 'DEN'), ('LAS', 'DEN'), ('LAS', 'ELP'), ('LAS', 'ELP'), ('LAS', 'HOU'), ('LAS', 'HOU'), ('LAS', 'LAX'), ('LAS', 'LAX'), ('LAS', 'LAX'), ('LAS', 'LAX'), ('LAS', 'LAX'), ('LAS', 'LIT'), ('LAS', 'MCI'), ('LAS', 'MCI'), ('LAS', 'MCI'), ('LAS', 'MDW'), ('LAS', 'MDW'), ('LAS', 'MDW'), ('LAS', 'MDW'), ('LAS', 'MDW'), ('LAS', 'OAK'), ('LAS', 'OAK'), ('LAS', 'OAK'), ('LAS', 'OAK'), ('LAS', 'OAK'), ('LAS',

In [16]:
OrigJoinRdd = flightOrigDestRdd.join(cityRdd)

print(OrigJoinRdd.take(5))

[('OAK', ('BOI', 'Oakland')), ('OAK', ('BUR', 'Oakland')), ('OAK', ('BUR', 'Oakland')), ('OAK', ('BUR', 'Oakland')), ('OAK', ('DEN', 'Oakland'))]


In [17]:
destOrigJoinRdd = OrigJoinRdd.join(cityRdd)

print(destOrigJoinRdd.take(5))

[('ONT', (('LAS', 'Ontario'), 'Ontario')), ('ONT', (('LAS', 'Ontario'), 'Ontario')), ('ONT', (('OAK', 'Ontario'), 'Ontario')), ('ONT', (('OAK', 'Ontario'), 'Ontario')), ('ONT', (('OAK', 'Ontario'), 'Ontario'))]


In [18]:
citiesCleanedRdd = destOrigJoinRdd.values()

print(citiesCleanedRdd.take(5))

[(('LAS', 'Ontario'), 'Ontario'), (('LAS', 'Ontario'), 'Ontario'), (('OAK', 'Ontario'), 'Ontario'), (('OAK', 'Ontario'), 'Ontario'), (('OAK', 'Ontario'), 'Ontario')]


In [19]:
citiesKV = citiesCleanedRdd.map(lambda Cities: (Cities, 1))

print(citiesKV.take(5))

[((('LAS', 'Ontario'), 'Ontario'), 1), ((('LAS', 'Ontario'), 'Ontario'), 1), ((('OAK', 'Ontario'), 'Ontario'), 1), ((('OAK', 'Ontario'), 'Ontario'), 1), ((('OAK', 'Ontario'), 'Ontario'), 1)]


In [20]:
citiesReducedSortedRdd = citiesKV.reduceByKey(lambda x,y: x+y)\
    .map(lambda x: (x[1], x[0]))\
    .sortByKey(ascending=False)

print(citiesReducedSortedRdd.take(5))

[(5540, (('BOS', 'NewYork'), 'NewYork')), (4056, (('LAX', 'SanFrancisco'), 'SanFrancisco')), (4010, (('ORD', 'NewYork'), 'NewYork')), (3905, (('DAL', 'Houston'), 'Houston')), (3895, (('DCA', 'NewYork'), 'NewYork'))]


#### 3. Find the longest departure delays (15 min or more)


In [21]:
file_path = 'file:///home/talentum/test-jupyter/test/PairRdd/flights.csv'

In [22]:
delayRdd = sc.textFile(file_path) \
    .map(lambda val: val.split(',')) \
    .filter(lambda delay: int(delay[11]) > 15) \
    .map(lambda column: (column[5], column[11]))

print(delayRdd.take(5))

[('WN', '25'), ('WN', '67'), ('WN', '87'), ('WN', '29'), ('WN', '82')]


In [23]:
delayMaxRdd = delayRdd.reduceByKey(lambda x,y: max(int(x), int(y)))

print(delayMaxRdd.take(5))

[('XE', 781), ('YV', 526), ('OH', 680), ('OO', 767), ('UA', 1268)]


#### 4. Remove the records that contain incomplete details



In [24]:
file_path = 'file:///home/talentum/test-jupyter/test/PairRdd/plane-data.csv' 

In [25]:
rdd4 = sc.textFile(file_path)

rdd4 = rdd4.filter(lambda line: len(line.split(',')) == expected_column_count)

print(rdd.take(5))

[['iata', 'airport', 'city', 'state', 'country', 'lat', 'long'], ['00M', 'Thigpen', 'BaySprings', 'MS', 'USA', '31.95376472', '-89.23450472'], ['00R', 'LivingstonMunicipal', 'Livingston', 'TX', 'USA', '30.68586111', '-95.01792778'], ['00V', 'MeadowLake', 'ColoradoSprings', 'CO', 'USA', '38.94574889', '-104.5698933'], ['01G', 'Perry-Warsaw', 'Perry', 'NY', 'USA', '42.74134667', '-78.05208056']]


In [26]:
planeDataRdd = sc.textFile(file_path)

print(planeDataRdd.count())

5030


In [27]:
cleanedPlaneDataRdd = planeDataRdd.map(lambda val: val.split(','))\
    .filter(lambda elements: len(elements) == 9)

print(cleanedPlaneDataRdd.count())

4481


$$
\Large\text{DataFrame}
$$

#### Finding the airlines with highest number of flights

In [28]:
file_path = 'file:///home/talentum/test-jupyter/test/PairRdd/flights.csv'

In [29]:
df = spark.read.csv(file_path)
df.show(5)

+---+---+---+----+----+---+----+------+---+---+----+----+----+----+----+----+----+----+----+----+
|_c0|_c1|_c2| _c3| _c4|_c5| _c6|   _c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|
+---+---+---+----+----+---+----+------+---+---+----+----+----+----+----+----+----+----+----+----+
|  1|  3|  4|2003|2211| WN| 335|N712SW|128|116| -14|   8| IAD| TPA| 810|   4|   8|   0|null|   0|
|  1|  3|  4| 926|1054| WN|1746|N612SW| 88| 78|  -6|  -4| IND| BWI| 515|   3|   7|   0|null|   0|
|  1|  3|  4|1940|2121| WN| 378|N726SW|101| 87|  11|  25| IND| JAX| 688|   4|  10|   0|null|   0|
|  1|  3|  4|1937|2037| WN| 509|N763SW|240|230|  57|  67| IND| LAS|1591|   3|   7|   0|null|   0|
|  1|  3|  4| 754| 940| WN|1144|N778SW|226|205| -15|   9| IND| PHX|1489|   5|  16|   0|null|   0|
+---+---+---+----+----+---+----+------+---+---+----+----+----+----+----+----+----+----+----+----+
only showing top 5 rows



In [30]:
carrier_counts = df.select(df._c5)\
    .groupBy('_c5')\
    .agg(F.count('*').alias('count'))\
    .orderBy(F.desc('count'))

carrier_counts.show(3)

+---+------+
|_c5| count|
+---+------+
| WN|356167|
| AA|175969|
| OO|166445|
+---+------+
only showing top 3 rows



#### 2. Determine the most common routes between two cities

In [31]:
airports_path = 'file:///home/talentum/test-jupyter/test/PairRdd/airports.csv'

In [32]:
airports_df = spark.read.csv(airports_path, header=True)
airports_df.show()

+----+--------------------+------------------+-----+-------+-----------+------------+
|iata|             airport|              city|state|country|        lat|        long|
+----+--------------------+------------------+-----+-------+-----------+------------+
| 00M|             Thigpen|        BaySprings|   MS|    USA|31.95376472|-89.23450472|
| 00R| LivingstonMunicipal|        Livingston|   TX|    USA|30.68586111|-95.01792778|
| 00V|          MeadowLake|   ColoradoSprings|   CO|    USA|38.94574889|-104.5698933|
| 01G|        Perry-Warsaw|             Perry|   NY|    USA|42.74134667|-78.05208056|
| 01J|     HilliardAirpark|          Hilliard|   FL|    USA| 30.6880125|-81.90594389|
| 01M|    TishomingoCounty|           Belmont|   MS|    USA|34.49166667|-88.20111111|
| 02A|          Gragg-Wade|           Clanton|   AL|    USA|32.85048667|-86.61145333|
| 02C|             Capitol|        Brookfield|   WI|    USA|   43.08751|-88.17786917|
| 02G|    ColumbianaCounty|     EastLiverpool|   OH|  

In [33]:
city_df = airports_df.select(
    F.col("iata").alias("iata"),
    F.col("city").alias("city")
)
city_df.show(5)

+----+---------------+
|iata|           city|
+----+---------------+
| 00M|     BaySprings|
| 00R|     Livingston|
| 00V|ColoradoSprings|
| 01G|          Perry|
| 01J|       Hilliard|
+----+---------------+
only showing top 5 rows



In [34]:
flights_path = 'file:///home/talentum/test-jupyter/test/PairRdd/flights.csv'

In [35]:
flights_df = spark.read.csv(flights_path)

In [36]:
flight_orig_dest_df = flights_df.select(
    F.col("_c12").alias("origin"),
    F.col("_c13").alias("destination")
)
flight_orig_dest_df.show(5)


+------+-----------+
|origin|destination|
+------+-----------+
|   IAD|        TPA|
|   IND|        BWI|
|   IND|        JAX|
|   IND|        LAS|
|   IND|        PHX|
+------+-----------+
only showing top 5 rows



In [46]:
origin_joined_df = flight_orig_dest_df.join(
    F.broadcast(city_df),
    flight_orig_dest_df.origin == city_df.iata,
    "inner"
).select("destination", "city").withColumnRenamed("city", "origin_city")


In [49]:
dest_origin_joined_df = origin_joined_df.join(
    F.broadcast(city_df),
    origin_joined_df.destination == city_df.iata,
    "inner"
).select("origin_city", "city").withColumnRenamed("city", "dest_city")

In [50]:
city_pairs_count = dest_origin_joined_df\
    .groupBy("origin_city", "dest_city")\
    .agg(F.count("*").alias("count"))\
    .orderBy(F.desc("count"))

city_pairs_count.show(5)

+------------+----------+-----+
| origin_city| dest_city|count|
+------------+----------+-----+
|     NewYork|    Boston| 5540|
|      Boston|   NewYork| 5478|
|     Chicago|   NewYork| 4103|
|SanFrancisco|LosAngeles| 4056|
|     NewYork|   Chicago| 4010|
+------------+----------+-----+
only showing top 5 rows



#### 3. Find the longest departure delays (15 min or more)

In [40]:
flights_df = spark.read.csv('file:///home/talentum/test-jupyter/test/PairRdd/flights.csv')

In [41]:
delays_df = flights_df.select(
    F.col("_c5").alias("flight_num"),
    F.col("_c11").cast(IntegerType()).alias("delay")
).filter(F.col("delay") > 15)
print("Delays DataFrame:")
delays_df.show(5)

Delays DataFrame:
+----------+-----+
|flight_num|delay|
+----------+-----+
|        WN|   25|
|        WN|   67|
|        WN|   87|
|        WN|   29|
|        WN|   82|
+----------+-----+
only showing top 5 rows



In [42]:
max_delays_df = delays_df.groupBy("flight_num").agg(F.max("delay").alias("max_delay")).orderBy("max_delay", ascending=False)
print("Maximum Delays DataFrame:")
max_delays_df.show(5)

Maximum Delays DataFrame:
+----------+---------+
|flight_num|max_delay|
+----------+---------+
|        NW|     2457|
|        AA|     1521|
|        MQ|     1295|
|        UA|     1268|
|        9E|     1099|
+----------+---------+
only showing top 5 rows



#### 4. Remove the records that contain incomplete details

In [43]:
file_path = 'file:///home/talentum/test-jupyter/test/PairRdd/plane-data.csv'

In [44]:
plane_data_df = spark.read.csv(
    file_path,
    header=True,
    inferSchema=True
)
print("Total plane records:", plane_data_df.count())

Total plane records: 5029


In [45]:
filtered_df = plane_data_df.filter("type IS NOT NULL")

filtered_df.count()

4480